# Solving Viscous Wave Equation

This notebook is used to solve the particular viscous wave equation for the setup of a muon passing through liquid Xenon. The goal is to find the pressure distribution in a container and its decay as a function of distance and time.

## Wave equation

We can derive the viscous wave equation through the linearised Navier-Stokes Equations. The full derivation is in the notes, but the point is that we have added a single term that takes into account the viscous effects. The equation turns out to be as follows:

$$\Delta \left(p(\vec{x},t) - \frac{1}{\omega_0}\frac{\partial}{\partial t}p(\vec{x},t)\right) = \frac{1}{c^2} \frac{\partial}{\partial}p(\vec{x},t)$$

where $c$ is the speed of the wave in the medium, $\omega_0$ is the coefficient responsible for damping, and $p(\vec{x},t)$ is pressure as a function of space and time.

We now introduce the linear differential operator $\mathcal{L}$.

$$\mathcal{L} := \Delta \left(p(\vec{x},t) - \frac{1}{\omega_0}\frac{\partial}{\partial t}p(\vec{x},t)\right) - \frac{1}{c^2} \frac{\partial}{\partial}p(\vec{x},t)$$

## Fundamental Solution

Now we search for the fundamental solution $F(\vec{x},t)$ that satisfies the follwing.

$$\mathcal{L}F(\vec{x},t) = \delta (\vec{x})\delta(t)$$

We can solve it using fourier methods and then we obtain the fundamental solution in terms of this integral over fourier momentum. Now we just need to evaluate this.

$$F(\vec{x},t) = \Theta(t) \int_0^\infty 8\pi^2 c^2 k^2 e^{-\frac{c^2 k^2}{2 \omega_0}t} \frac{e^{i k r} - e^{-ikr}}{2ikr} \frac{e^{ikct\sqrt{1-\frac{k^2c^2}{4\omega_0}}} - e^{-ikct\sqrt{1-\frac{k^2c^2}{4\omega_0}}}}{2ikct\sqrt{1-\frac{k^2c^2}{4\omega_0}}}dk$$

Where $\Theta(t)$ is the Heaviside function and $r:=\left| \vec{x}\right|$

## Integration

To numerically evaluate this expression we first need to study it. Therefore let's define the function f as the inner part of the integral like so.

$$F(\vec{x},t) = \Theta(t)\int_0^\infty f(k;\vec{x},t)dk$$

Now we realise that the function f is a piecewise function plotted below.

In [3]:
#import relevant libraries
import numpy as np
import scipy.constants as const
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from tqdm import tqdm

# if this doesn't work run: "python3 -m pip install ipympl" 
# or just comment it out and learn to live without sliders on the graphs
%matplotlib widget       

# Define the relevant constants
c   = 1    # Speed of wave
w_0 = 1    # Damping frequency

# Define integrable function f
def f(k,r,t,w_0=w_0,c=c):
    if t < 0:
        return 0
    
    if k <= 0:
        return 0;
    
    if k < 2*w_0/c:
        return 8 * np.pi**2 * c**2 * k**2 * np.exp(-c**2 * k**2 * t / (2 * w_0)) *\
                np.sinc(k*c*t/np.pi *(1 - k**2 * c**2 / (4 * w_0**2))**0.5) *\
                np.sin(r*k)/(r*k)
    
    elif k > 2*w_0/c:
        return 8 * np.pi**2 * c**2 * k**2 *\
                (np.exp(k*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 - c**2 * k**2 * t / (2 * w_0)) -\
                np.exp(-k*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 - c**2 * k**2 * t / (2 * w_0)))/\
                (2*k*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 ) * np.sin(r*k)/(r*k)
    else: return 0
    
# Define the envelope function of f
def e(k,r,t):
    if t < 0:
        return 0
    
    if k <= 0:
        return 0;
    
    if k < 2*w_0/c:
        return 8 * np.pi**2 * c**2 * np.exp(-c**2 * k**2 * t / (2 * w_0))/\
                (r*c*t *(1 - k**2 * c**2 / (4 * w_0**2))**0.5)
    
    elif k > 2*w_0/c:
        return 8 * np.pi**2 * c**2 *\
                (np.exp(k*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 - c**2 * k**2 * t / (2 * w_0)) -\
                np.exp(-k*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 - c**2 * k**2 * t / (2 * w_0)))/\
                (2*c*t *(-1 + k**2 * c**2 / (4 * w_0**2))**0.5 )/r
    else: return 0

In [4]:
# Set up the plot
fig = plt.figure(figsize=(9,6))
fig.suptitle("Integrand plot vs k")
ax = fig.add_subplot(111)

k_min = 0
k_max = 100
Npts = 100

ax.set_xlim([k_min, k_max])
ax.set_ylabel(r'$f(k;r,t)$',fontsize = 15)
ax.set_xlabel(r'$k$',fontsize = 15)
ax.grid(True)

# Generate x-y values
k = np.linspace(k_min,k_max,Npts)
def f_vect(K,r,t):
    return np.array([f(k,r,t) for k in K])

# Add the sliders
@widgets.interact(r=(0.001, 100, 0.1), t=(0.001, 10, .01), Npts=(100,10000, 1))
def update(r =1.0, t=1.0, Npts=1000):
    [l.remove() for l in ax.lines]
    k = np.linspace(k_min,k_max,Npts)
    ax.plot(k, f_vect(k,r,t), color='C3')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

interactive(children=(FloatSlider(value=1.0, description='r', min=0.001), FloatSlider(value=1.0, description='…

# Integration

Now that we have the plot we need to somehow integrate it. To do this efficiently we need to find when does the wave decay over a specific amount. As a result we need to find its envelope. To do this we have to study each of the pieces of the piecewise function seperately.

## Envelope

Now based on the analytic expression of the integrand we can conclude that the envelope function $\varepsilon(k;r,t)$ is the following.

$$
\varepsilon(k;r,t) = \begin{cases}
\frac{8\pi^2 c^2 e^{-\frac{c^2 k^2}{2 \omega_0}t}}{rct\sqrt{1-\frac{k^2c^2}{4\omega_0}}} & \text{if } k \leq \frac{2\omega_0}{c}\\ 
8\pi^2 c^2 e^{-\frac{c^2 k^2}{2 \omega_0}t}\frac{\text{sinh}\left(kct\sqrt{\frac{k^2c^2}{4\omega_0}-1}\right)}{rct\sqrt{\frac{k^2c^2}{4\omega_0}-1}} & \text{if } k > \frac{2\omega_0}{c}\\ 
\end{cases}
$$

Let's try to plot one against the other

In [5]:
# Set up the plot
fig2 = plt.figure(figsize=(9,6))
fig2.suptitle("Integrand plot vs k with envelope")
ax2 = fig2.add_subplot(111)

k_min = 0
k_max = 100
Npts = 100

ax2.set_ylim([-10, 40])
ax2.set_xlim([k_min, k_max])
ax2.set_ylabel(r'$f(k;r,t)$',fontsize = 15)
ax2.set_xlabel(r'$k$',fontsize = 15)
ax2.grid(True)

# Generate x-y values
k2 = np.linspace(k_min,k_max,Npts)
def f_vect(K,r,t):
    return np.array([f(k,r,t) for k in K])

def e_vect(K,r,t):
    return np.array([e(k,r,t) for k in K])

# Add the sliders
@widgets.interact(r=(0.001, 100, 0.1), t=(0.001, 10, .01), Npts=(100,10000, 1))
def update2(r = 1.0, t=1.0, Npts=1000):
    [l.remove() for l in ax2.lines]
    k2 = np.linspace(k_min,k_max,Npts)
    ax2.plot(k2, f_vect(k2,r,t), color='C3')
    ax2.plot(k2, e_vect(k2,r,t), color='C0')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

interactive(children=(FloatSlider(value=1.0, description='r', min=0.001), FloatSlider(value=1.0, description='…

## Approximations

Now to analytically calculate the integral of that equation we need to make certain approximations. To do this we need some intuitive understanding of the order of magnitude of the constants we are using. 

### Constants

We have two defining constants in the problem. The decay frequency $\omega_0$ and the speed of sound in LXe $c$. From the wave equation we have derived that the speed of sound is:

$$c = \sqrt{\frac{K}{\rho_0}}$$

Where $K$ is the bulk modulus and $\rho_0$ is the rest density of Liquid Xenon. Now the biggest question comes from trying to find values for these constants. From [nist](https://webbook.nist.gov/cgi/fluid.cgi?ID=C7440633&Action=Page) we obtain the fololowing list of constants:

- Bulk Modulus $K$ $[\frac{kg}{s^2 m}]$: $1.2667 \times 10^9$
- Rest Density $\rho_0$ $[\frac{kg}{m^3}]$: $2.9663 \times 10^3$

Therefore we calculate the speed of sound in LXe to be:

$$c = \sqrt{\frac{K}{\rho_0}} = 653.47 \frac{m}{s}$$

Now for the attenuation frquency $\omega_0$ we have derived in the equation that it is the ratio of the Bulk modulus to the viscocity. Similartly to the speed of sound we have obtained the following values from [nist](https://webbook.nist.gov/cgi/fluid.cgi?ID=C7440633&Action=Page).

- Bulk Modulus $K$ $[\frac{kg}{s^2 m}]$: $1.2667 \times 10^9$
- viscocity $\mu$ $[\text{Pa}s]$: $5.1701 \times 10^{-4}$

We therefore obtain that the attenuation frequency is:

$$\omega_0 = \frac{K}{\mu} = 2.4500 \times 10^{12} $$

Now that we have the values we can start estimating the possible evolution of the system, by using some very useful approximations.


### Near... Far... wherever you are!

We see that there is a special point in $f$ at which the root becomes imaginary. That happens when $k = \frac{2\omega_0}{c}$. If we evaluate we see that the graph will be attenuated before that point. Therefore we can approximate the function by just considering the following case where $k \leq \frac{2 \omega_0}{c}$. We obtain the following expression for $f(k;r,t)$.

$$f(k;r,t)=\frac{8\pi^2 c e^{-\frac{c^2 k^2}{2 \omega_0}t}}{rt\sqrt{1-\frac{k^2c^2}{4\omega_0}}} \sin(k r) \sin \left(kct\sqrt{1-\frac{k^2c^2}{4\omega_0}}\right) $$

Now that's a way prettier equation. However it is still a pain in the *** to integrate. Therefore we can consider one more simplification. Let's look at the taylor series of the following function in x centered at 0 $\forall \alpha > 0$.

$$ \sqrt{1 - \alpha x^2} = 1 - \frac{\alpha x^2}{2} - \frac{\alpha^2 x^4}{8} + \mathcal{O}(x^6)$$

in our case we notice how tiny $\alpha$ is and hence we drop all the terms except the first in our expansion. Finally we obtain a much more manageble expression for the integral in question. We call that new function $\phi$.

$$f(k;r,t) \approx \phi(k;r,t) = \frac{8\pi^2 c}{rt} e^{-\frac{c^2 k^2}{2 \omega_0}t} \sin(k r) \sin \left(kct\right)$$

In fact we can plot both $f(k;r,t)$ and $\phi(k;r,t)$ to see their differences.